In [1]:
import time
import pickle
import pandas as pd
import numpy as np

with open('strava/access_token.pickle', 'rb') as f:
    access_token = pickle.load(f)

MY_STRAVA_CLIENT_ID, MY_STRAVA_CLIENT_SECRET, token = open('strava/client.secret').read().strip().split(',')

from stravalib.client import Client

client = Client()

if time.time() > access_token['expires_at']:
    print('Token has expired, will refresh')
    refresh_response = client.refresh_access_token(client_id=MY_STRAVA_CLIENT_ID, 
                                               client_secret=MY_STRAVA_CLIENT_SECRET, 
                                               refresh_token=access_token['refresh_token'])
    access_token = refresh_response
    with open('../access_token.pickle', 'wb') as f:
        pickle.dump(refresh_response, f)
    print('Refreshed token saved to file')

    client.access_token = refresh_response['access_token']
    client.refresh_token = refresh_response['refresh_token']
    client.token_expires_at = refresh_response['expires_at']
        
else:
    print('Token still valid, expires at {}'
          .format(time.strftime("%a, %d %b %Y %H:%M:%S %Z", time.localtime(access_token['expires_at']))))

    client.access_token = access_token['access_token']
    client.refresh_token = access_token['refresh_token']
    client.token_expires_at = access_token['expires_at']

ERROR:root:Please make sure your STRAVA_CLIENT_ID is set in your environment.


Token has expired, will refresh


Refreshed token saved to file


In [2]:
athlete = client.get_athlete()
print("Athlete's name is {} {}, based in {}, {}"
      .format(athlete.firstname, athlete.lastname, athlete.city, athlete.country))

Athlete's name is Lucas Menger, based in Frankfurt am Main, Germany


In [3]:
activities = client.get_activities(limit=1000)
list(activities)[0:10]

[SummaryActivity(bound_client=<stravalib.client.Client object at 0x0000023C39EF8DD0>, id=13706990055, achievement_count=3, athlete=MetaAthlete(bound_client=None, id=35616673, resource_state=1), athlete_count=2, average_speed=1.813, average_watts=89.8, comment_count=0, commute=False, device_watts=False, distance=13323.9, elapsed_time=11608, elev_high=431.9, elev_low=230.6, end_latlng=LatLon(root=[49.792068, 8.668235]), external_id='garmin_ping_413607775571', flagged=False, gear_id='b10756923', has_kudoed=False, hide_from_home=None, kilojoules=660.1, kudos_count=0, manual=False, map=Map(id='a13706990055', polyline=None, summary_polyline='ct{nHgv{s@tAdAl@RbEZrBiAl@o@|BiF\\UrACl@Wv@A~B]rAf@`AH^g@~@Ul@@t@w@PHWLQAIZ]JmAGe@UEE?w@He@QAi@mBKe@Cm@QYUEcAoAe@[w@Ow@Py@OcA}AUGGYRCdBu@^CdAVjBBhAOzA`@nBVn@^dAz@x@Er@P\\Tp@Gr@LZ^l@bAxAnAbBNhAIzAa@|@@vDtAh@^zAdBjA|Av@nAJd@hC|G\\lBFn@ObCD_@DCNXLhA|@OZXVBd@VHb@d@r@FrAlKd@XINw@KRAE\\Un@Lv@h@lEwAf@Er@]tAwA~@I~@b@ZF~@E`Ac@`@zABt@Ch@Kb@}AtA_CJuACmAT{AvA}AhAi@R

In [16]:
data = []
for activity in activities:
    tmp = {
        'type': activity.type,
        'name': activity.name,
        'start_date_local': activity.start_date_local,
        'moving_time': activity.moving_time,
        'elapsed_time': activity.elapsed_time,
        'average_heartrate': activity.average_heartrate,
        'max_heartrate': activity.max_heartrate,
    }
    data.append(tmp)

df = pd.DataFrame(data)

In [17]:
df['start_date_local'] = pd.to_datetime(df['start_date_local'])
df

,type,name,start_date_local,moving_time,elapsed_time,average_heartrate,max_heartrate
0,root='Ride',Afternoon Ride,2025-02-23 14:14:46+00:00,7348,11608,137.9,160.0
1,root='Ride',Afternoon Mountain Bike Ride,2025-02-23 14:14:38+00:00,7898,11662,147.2,176.0
2,root='Ride',Lunch Mountain Bike Ride,2025-02-23 11:08:41+00:00,1360,1551,118.6,150.0
3,root='RockClimbing',Uni Bouldern,2025-02-21 18:17:06+00:00,6802,6802,130.8,165.0
4,root='Ride',Afternoon Mountain Bike Ride,2025-02-19 16:37:45+00:00,2462,2683,153.9,177.0
...,...,...,...,...,...,...,...
567,root='Ride',Radfahrt am Nachmittag,2020-04-29 14:13:53+00:00,259,622,NaN,NaN
568,root='Ride',Radfahrt am Mittag,2020-04-29 12:16:59+00:00,1420,6880,NaN,NaN
569,root='Ride',Saas,2020-04-22 13:55:42+00:00,592660,598825,NaN,NaN
570,root='Ride',Radfahrt am Mittag,2020-04-22 12:35:37+00:00,1507,2016,NaN,NaN


In [19]:
df['end_date_local'] = df['start_date_local'] + pd.to_timedelta(df['elapsed_time'], unit='s')

In [21]:
# filter for activities end after 17:00
df_filter = df[df['end_date_local'].dt.hour >= 17]